In [1]:
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score



In [2]:

from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from keras.preprocessing import text, sequence

import nltk
import re
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [3]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [10]:
##TODO

csv_file = '/Users/swcam/Documents/GitHub/Sarcasm/Final_data/liwc_response_context_train.csv'
#csv_file = '/Users/swcam/Documents/GitHub/Sarcasm/Final_Features_train_data'
Train_df = pd.read_csv(csv_file)
Train_df = Train_df.loc[:, ~Train_df.columns.str.contains('^Unnamed')]
# print out the first few rows of data info
Train_df.head(5)

#These are the most useful features per SHAP 


##ALL
#Train_df_select = Train_df[['vader_compound','vader_pos','Emotional_sim_response/context/0','you','noun_count_percent','upper_case_word_count','char_count','adv_count_percent','time','context1_emotion','incl','cos_sim','pro_count_percent','ipron','vader_neg','context0_emotion','dom_diff_c0_resp','Emotional_sim_response/context/1','response_emotion','aro_diff_c1_resp']].copy()

Train_df_select = Train_df[['time', 'achieve','article','preps','relativ','funct']]
##Below per SHAP on only the Training data
#Train_df_select = Train_df[['vader_compound','pro_count_percent','vader_pos','vader_neg','context0_emotion','noun_count_percent','time','char_count','adv_count_percent','you','cos_sim1','vader_neu','adj_count_percent','cos_sim','context1_emotion','pronoun','incl','discrep','upper_case_word_count','negate','val_diff_c1_resp','aro_diff_c1_resp','dom_diff_c1_resp']].copy()

##Below per SHAP on the TEST and Trainind data
#Train_df_select = Train_df[['friend','vader_compound','response_emotion','word_density','cos_sim','char_count','context0_emotion','vader_neg','relativ','ingest','cos_sim1','humans','punctuation_count','context1_emotion','upper_case_word_count','i','ppron','pro_count_percent','cogmech','article']]
##Top few
#Train_df_select = Train_df[['vader_compound', 'context0_sentiment', 'word_density','posemo','work','cos_sim1','you','ppron','Response_emotion','ipron','vader_neu','cos_sim','punctuation_count','char_count','negemo','noun_count_percent']].copy()
#Train_df_select = Train_df[['vader_compound','response_emotion','word_density','cos_sim','context0_emotion','vader_neg','cos_sim1','humans','punctuation_count','context1_emotion','upper_case_word_count','ppron','pro_count_percent','article']]


In [11]:
##TODO

csv_file = '/Users/swcam/Documents/GitHub/Sarcasm/Final_data/liwc_response_context_test.csv'
#csv_file = '/Users/swcam/Documents/GitHub/Sarcasm/Final_Features_test_data'
Test_df = pd.read_csv(csv_file)
Test_df = Test_df.loc[:, ~Test_df.columns.str.contains('^Unnamed')]
# print out the first few rows of data info
Test_df.head(5)

# These are the most useful features per SHAP
#ALL
#Test_df_select = Test_df[['vader_compound','pro_count_percent','vader_pos','vader_neg','context0_emotion','noun_count_percent','time','char_count','adv_count_percent','you','cos_sim1','vader_neu','adj_count_percent','cos_sim','context1_emotion','pronoun','incl','discrep','upper_case_word_count','negate']].copy()

## PEr SHAP on just the TRAIN data
#Test_df_select = Test_df[['vader_compound','vader_pos','Emotional_sim_response/context/0','you','noun_count_percent','upper_case_word_count','char_count','adv_count_percent','time','context1_emotion','incl','cos_sim','pro_count_percent','ipron','vader_neg','context0_emotion','dom_diff_c0_resp','Emotional_sim_response/context/1','response_emotion','aro_diff_c1_resp']].copy()
Test_df_select = Test_df[['time', 'achieve','article','preps','relativ','funct']]
## Per SHAP on the TEST and TRAIN data
#Test_df_select = Test_df[['friend','vader_compound','response_emotion','word_density','cos_sim','char_count','context0_emotion','vader_neg','relativ','ingest','cos_sim1','humans','punctuation_count','context1_emotion','upper_case_word_count','i','ppron','pro_count_percent','cogmech','article']]

#Test_df_select = Test_df[['vader_compound','response_emotion','word_density','cos_sim','context0_emotion','vader_neg','cos_sim1','humans','punctuation_count','context1_emotion','upper_case_word_count','ppron','pro_count_percent','article']]

##Top few
#Test_df_select = Test_df[['vader_compound', 'context0_sentiment', 'word_density','posemo','work','cos_sim1']].copy()

In [12]:
X_train = Train_df_select

X_test= Test_df_select

y_train= Train_df['label']

y_test = Test_df['label']

In [ ]:

X_test['response_emotion'] = X_test['response_emotion'].astype('category').cat.codes
X_test['context1_emotion'] = X_test['context1_emotion'].astype('category').cat.codes
X_test['context0_emotion'] = X_test['context0_emotion'].astype('category').cat.codes



X_train['context1_emotion'] = X_train['context1_emotion'].astype('category').cat.codes
X_train['response_emotion'] = X_train['response_emotion'].astype('category').cat.codes
X_train['context0_emotion'] = X_train['context0_emotion'].astype('category').cat.codes

In [13]:

from sklearn import preprocessing
encoder = preprocessing.LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.fit_transform(y_test)

In [14]:
X_test.shape

(1800, 6)

In [15]:
X_train.shape

(5000, 6)

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
# fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)

# predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)

    return metrics.accuracy_score(predictions, y_test),metrics.precision_score(predictions, y_test),metrics.recall_score(predictions, y_test),metrics.f1_score(predictions, y_test)


# Results on Test data

# Naive Bayes


In [17]:
from sklearn.naive_bayes import GaussianNB

accuracy, precision, recall,f1  = train_model(naive_bayes.GaussianNB(), X_train, y_train, X_test)
print ("NB, : ", "A:", round(accuracy,2), "P:", round(precision,2), "R:", round(recall,2), "F1:", round(f1,2))

NB, :  A: 0.52 P: 0.6 R: 0.52 F1: 0.56


# Random Forest

In [18]:
from sklearn.ensemble import RandomForestClassifier
accuracy, precision, recall,fl  = train_model(RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0), X_train, y_train, X_test)
print ("random Forest: ", "A:", round(accuracy,2), "P:", round(precision,2), "R:", round(recall,2), "F1:", round(f1,2))

random Forest:  A: 0.52 P: 0.45 R: 0.53 F1: 0.56


# SVM

In [19]:
from sklearn.svm import SVC

accuracy, precision, recall,f1  = train_model(svm.SVC(), X_train, y_train, X_test)
print ("Svm: ", "A:", round(accuracy,2), "P:", round(precision,2), "R:", round(recall,2), "F1:", round(f1,2))

Svm:  A: 0.52 P: 0.45 R: 0.52 F1: 0.48


# LogisticRegression

In [20]:


accuracy, precision, recall,f1  = train_model(LogisticRegression(random_state=0,max_iter=500), X_train, y_train, X_test)
print ("Logistic Regression: ", "A:", round(accuracy,2), "P:", round(precision,2), "R:", round(recall,2), "F1:", round(f1,2))

Logistic Regression:  A: 0.53 P: 0.49 R: 0.53 F1: 0.51


# Test on Train data


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score

models = [
    RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0),
    LinearSVC(),
    MultinomialNB(),
    LogisticRegression(random_state=0,max_iter=500),
    GaussianNB(),
    svm.SVC(),
]
CV = 10
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []
for model in models:
  model_name = model.__class__.__name__
  accuracies = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=CV)
  for fold_idx, accuracy in enumerate(accuracies):
    entries.append((model_name, fold_idx, accuracy))
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])
import seaborn as sns
sns.boxplot(x='model_name', y='accuracy', data=cv_df)
sns.stripplot(x='model_name', y='accuracy', data=cv_df, 
              size=8, jitter=True, edgecolor="gray", linewidth=2)
plt.show()

In [ ]:
cv_df.groupby('model_name').accuracy.mean()

In [ ]:
cv_df.groupby('model_name').accuracy.max()

In [ ]:
cv_df.groupby('model_name').accuracy.min()